In [ ]:
#coding=utf8  
#!/usr/bin/python  
import struct,socket  
import hashlib  
import threading,random  
import time  
import struct  
from base64 import b64encode, b64decode  

connectionlist = {}  
g_code_length = 0 
g_header_length = 0 
   
def hex2dec(string_num):  
    return str(int(string_num.upper(), 16))  
   
def get_datalength(msg):  
    global g_code_length  
    global g_header_length      
       
    print ("get data length :",len(msg))
    g_code_length = msg[1] & 127 
    received_length = 0;  
    if g_code_length == 126:
        #g_code_length = msg[2:4]  
        #g_code_length = (ord(msg[2])<<8) + (ord(msg[3]))  
        g_code_length = struct.unpack('>H', msg[2:4])[0]  
        g_header_length = 8 
    elif g_code_length == 127:  
        #g_code_length = msg[2:10]  
        g_code_length = struct.unpack('>Q', msg[2:10])[0]  
        g_header_length = 14 
    else:  
        g_header_length = 6 
    g_code_length = int(g_code_length)  
    return g_code_length  
           
def parse_data(msg):  
    global g_code_length  
    g_code_length = msg[1] & 127 
    received_length = 0;  
    if g_code_length == 126:  
        g_code_length = struct.unpack('>H', msg[2:4])[0]  #
        masks = msg[4:8]  
        data = msg[8:]  
    elif g_code_length == 127:  
        g_code_length = struct.unpack('>Q', msg[2:10])[0]  #
        masks = msg[10:14]  
        data = msg[14:]  
    else:  
        masks = msg[2:6]  
        data = msg[6:]  
        
    i = 0 
    raw_str = ''  
    for d in data:  
        raw_str += chr(d ^ masks[i%4])  
        i += 1 
    print (u"总长度是：%d" % int(g_code_length))      
    return raw_str    
   
def sendMessage(message):  
    global connectionlist  
    print("sendMessage")
    message_utf_8 = message.encode('utf-8') 
    print('message_utf_8 :',message_utf_8)
    for connection in connectionlist.values():  
        back_str = []  
        back_str.append(b'\x81')  
        data_length = len(message_utf_8)  
        print("The length of message is",data_length)
   
        if data_length <= 125:  
            back_str.append(struct.pack('b',data_length))  
        elif data_length <= 65535 :
            back_str.append(struct.pack('b', 126))  
            back_str.append(struct.pack('>h', data_length))  
            #back_str.append(chr(data_length >> 8))  
            #back_str.append(chr(data_length & 0xFF))  
            #a = struct.pack('>h', data_length)  
            #b = chr(data_length >> 8)  
            #c = chr(data_length & 0xFF)  
        elif data_length <= (2^64-1):  
            #back_str.append(chr(127))  
            back_str.append(struct.pack('b', 127))  
            back_str.append(struct.pack('>q', data_length))  
            #back_str.append(chr(data_length >> 8))  
            #back_str.append(chr(data_length & 0xFF))        
        else :  
                print (u'it is too long to send in a time')          
        msg = bytes()
        for c in back_str:  
            msg += c;  
        back_str = msg + message_utf_8
        #connection.send(str.encode(str(u"\x00%s\xFF\n\n" % message))) #这个是旧版  
        #print (u'send message:' +  message)  
        if back_str != None and len(back_str) > 0:  
            print ("it is going to send ",back_str)  
            connection.send(back_str)  
   
   
def deleteconnection(item):  
    global connectionlist  
    del connectionlist['connection'+item]  
   
   
class WebSocket(threading.Thread):#继承Thread  
   
    GUID = "258EAFA5-E914-47DA-95CA-C5AB0DC85B11" 
   
    def __init__(self,conn,index,name,remote, path="/"):  
        threading.Thread.__init__(self)#初始化父类Thread  
        self.conn = conn  
        self.index = index  
        self.name = name  
        self.remote = remote  
        self.path = path  
        self.buffer = ""  
        self.buffer_utf8 = ""  
        self.length_buffer = 0 
    def run(self):#重载Thread的run  
        print('Socket%s Start!' % self.index)  
        headers = {}  
        self.handshaken = False 
        while True:
            global g_code_length  
            global g_header_length
            
            if self.handshaken == False:
                print ('Socket%s Start Handshaken with %s!' % (self.index,self.remote))  
                rec=self.conn.recv(1024)
                self.buffer += bytes.decode(rec)
                print("Handshaken packages from client:",rec)
                if self.buffer.find('\r\n\r\n') != -1:
                    header, data = self.buffer.split('\r\n\r\n', 1)  
                    for line in header.split("\r\n")[1:]:  
                        key, value = line.split(": ", 1)  
                        headers[key] = value  
   
                    headers["Location"] = ("ws://%s%s" %(headers["Host"], self.path))  
                    key = headers['Sec-WebSocket-Key']  
                    token = b64encode(hashlib.sha1(str.encode(str(key + self.GUID))).digest())  
   
   
                    handshake="HTTP/1.1 101 Switching Protocols\r\n"\
                        "Upgrade: websocket\r\n"\
                        "Connection: Upgrade\r\n"\
                        "Sec-WebSocket-Accept: "+bytes.decode(token)+"\r\n"\
                        "WebSocket-Origin: "+str(headers["Origin"])+"\r\n"\
                        "WebSocket-Location: "+str(headers["Location"])+"\r\n\r\n" 
   
   
                    self.conn.send(str.encode(str(handshake)))  
                    self.handshaken = True   
                    print ('Socket %s Handshaken with %s success!' %(self.index, self.remote))    
                    sendMessage(u'Welcome, ' + self.name + ' !')    
                    self.buffer_utf8 = ""  
                    g_code_length = 0                     
   
   
            else:
                print("ready to recv data, please wait...")
                recv=self.conn.recv(128)  
                print("received data:",recv)
                if len(recv) <= 0:  
                    print('length of this data is',len(recv))
                    continue 
                if g_code_length == 0:  
                    get_datalength(recv)
                #接受的长度
                self.length_buffer = self.length_buffer + len(recv)  
                
                self.buffer = self.buffer.encode('utf-8') + recv
                if self.length_buffer - g_header_length < g_code_length :  
                    continue 
                else :  
                    self.buffer_utf8 = parse_data(self.buffer) #utf8      
                    print('The data has been parsed to',self.buffer_utf8)
                    print()
                    msg_unicode = self.buffer_utf8#.decode('utf-8', 'ignore') #unicode  
                    if msg_unicode=='quit':  
                        print (u'Socket%s Logout!' % (self.index))  
                        nowTime = time.strftime('%H:%M:%S',time.localtime(time.time()))  
                        sendMessage(u'%s %s say: %s' % (nowTime, self.remote, self.name+' Logout'))                        
                        deleteconnection(str(self.index))  
                        self.conn.close()  
                        break #退出线程  
                    else:  
                        #print (u'Socket%s Got msg:%s from %s!' % (self.index, msg_unicode, self.remote))  
                        nowTime = time.strftime(u'%H:%M:%S',time.localtime(time.time()))  
                        sendMessage(u'%s %s say: %s' % (nowTime, self.remote, msg_unicode))    
                    #重置buffer和bufferlength  
                    self.buffer_utf8 = ""  
                    self.buffer = ""  
                    g_code_length = 0 
                    self.length_buffer = 0 
            self.buffer = ""  
            print("###################")
            print('\n')
        print("out run()")
   
   
class WebSocketServer(object):  
    def __init__(self):  
        self.socket = None 
    def begin(self):  
        print( 'WebSocketServer Start!')  
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
        self.socket.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)  
        self.socket.bind(("0.0.0.0",12345))  
        self.socket.listen(50)  
   
   
        global connectionlist  
   
   
        i=0 
        while True:  
            connection, address = self.socket.accept()  
   
   
            username=address[0]       
            newSocket = WebSocket(connection,i,username,address)  
            newSocket.start() #开始线程,执行run函数  
            connectionlist['connection'+str(i)]=connection  
            i = i + 1 
            print('i=',i)
            print('\n')
            print('-------------------------------------------------------------------------')
            print('\n')
   
if __name__ == "__main__":  
    server = WebSocketServer()  
    server.begin() 

WebSocketServer Start!
Socket0 Start!i= 1


-------------------------------------------------------------------------



Socket0 Start Handshaken with ('127.0.0.1', 3459)!
Handshaken packages from client: b'GET / HTTP/1.1\r\nHost: 127.0.0.1:12345\r\nConnection: Upgrade\r\nPragma: no-cache\r\nCache-Control: no-cache\r\nUpgrade: websocket\r\nOrigin: http://127.0.0.1:14884\r\nSec-WebSocket-Version: 13\r\nUser-Agent: wechatdevtools appservice port/37708 token/f1ee2bde43de59356f533ef35a322f50\r\nAccept-Encoding: gzip, deflate, br\r\nAccept-Language: zh-CN,zh;q=0.8\r\nReferer: https://servicewechat.com/wx3f8720e8922856ff/devtools/page-frame.html\r\nSec-WebSocket-Key: S0z8am1YFAY3QokVOxanKw==\r\nSec-WebSocket-Extensions: permessage-deflate; client_max_window_bits\r\n\r\n'
Socket 0 Handshaken with ('127.0.0.1', 3459) success!
sendMessage
message_utf_8 : b'Welcome, 127.0.0.1 !'
The length of message is 20
it is going to send  b'\x81\x14Welcome, 127.0.0.1 !'
###################


ready to recv d

Exception in thread 127.0.0.1:
Traceback (most recent call last):
  File "D:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "<ipython-input-1-6e4e24e054a0>", line 163, in run
    recv=self.conn.recv(128)
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。



In [1]:
(b'0x81').decode('utf-8')

'0x81'

In [4]:
def parse_data(msg):
    v = msg[1] & 0x7f
    if v == 0x7e:
        p = 4
    elif v == 0x7f:
        p = 10
    else:
        p = 2
    p=2
    mask = msg[p:p + 4]
    data = msg[p + 4:]

    return ''.join([chr(v ^ mask[k % 4]) for k, v in enumerate(data)])

m=b'\x88\x9a/\xc3s\xe8,)>\x89\\\xa8\x16\x8c\x0f\xa5\x01\x89B\xa6S\x8e]\xac\x1e\xc8\\\xa6\x01\x9eJ\xb1'
# m=b'\x88\x9a#\x82\x0f\x01 hB`P\xe9je\x03\xe4}`N\xe7/gQ\xedb!P\xe7}wF\xf0'
print(parse_data(m))

êMasked frame from server


In [8]:
'a'.encode('utf-8')+'a'.encode('utf-8')

b'aa'

In [7]:
print(type(u'ni'))

<class 'str'>


In [1]:
0x12

18

In [8]:
print(1,'\n',2,'\n',3)

1 
 2 
 3


In [3]:
mm =b'\x81\x86\x98\x06\x1c\xe1\xaany\x8d\xf4i'
len(mm)

12